# Creating a repository from DMTF Schema

In addition to creating a repository specifically defined classes, etc. the repository can be created using the DMTF schema as the source for qualifier declarations and classes.

Using this method requires that you make the following decisions:

1. Which version of the DMTF released schema you intend to use.

2. Where in your local environment do you want to save and expand the schema. This should be a directory and serves both as the storage space for download and expansion and the location checked before download so that the schema is not downloaded each time it is used.

3. Use of the experimental schema for the defined DMTF schema release as well as the released schema.

4. The leaf classes you need. The method takes responsibility for determining superclasses and other required classes ( ex. associations, embedded instances) defined in the mof for any of the leaf classes you specify.

In [ ]:
import pywbem
import pywbem_mock

import os
print(os.getcwd())

# Defines the version of the DMTF schema to be installed.
# This demo is done with the schema already loaded into the pywbem development environment assuming that
# the user is using the docs/notebooks directory. This may change as pywbem updates the schema use
# in out tests
DMTF_TEST_SCHEME_VER = (2, 49, 0)
# location of schema 2.49.0 in pywbem 0.13.0 development
# code cloed from github
SCHEMA_DIR = os.path.join('..', '..', 'tests', 'schema')
print(SCHEMA_DIR)

# An alternative would be to define your own schema and schema directory
#DMTF_TEST_SCHEME_VER = "2.51.0"
#SCHEMA_DIR = "."

classnames = ['CIM_RegisteredProfile', 'CIM_Namespace', 'CIM_ObjectManager',
              'CIM_ElementConformsToProfile', 'CIM_ReferencedProfile']

conn = pywbem_mock.FakedWBEMConnection(default_namespace='root/cimv2')
print(conn)

conn.compile_dmtf_schema(DMTF_TEST_SCHEME_VER, SCHEMA_DIR,
                                 class_names=classnames, verbose=False)
conn.display_repository()

## Add some more classes to this repository

TODO

In [ ]:
mof= '''
    class TST_Class1 {
          [Key]
        string InstanceID;
        string Prop1;
    };

    class TST_Class2 {
          [Key]
        string InstanceID;
        string Prop2;
    };

      [Association]
    class TST_Association12 {
          [Key]
        TST_Class1 REF Ref1;
          [Key]
        TST_Class2 REF Ref2;
    };
'''

conn.compile_mof_string(mof)
print(conn.GetClass("TST_Class1"))

## adding more objects

In [ ]:
c1_key = pywbem.CIMProperty('InstanceID', type='string', value='111')
c1_path = pywbem.CIMInstanceName(
    'TST_Class1',
    keybindings={c1_key.name: c1_key.value},
)
c1 = pywbem.CIMInstance(
    'TST_Class1',
    properties=[
        c1_key,
        pywbem.CIMProperty('Prop1', type='string', value='1'),
    ],
    path=c1_path,
)

c2_key = pywbem.CIMProperty('InstanceID', type='string', value='222')
c2_path = pywbem.CIMInstanceName(
    'TST_Class2',
    keybindings={c2_key.name: c2_key.value},
)
c2 = pywbem.CIMInstance(
    'TST_Class2',
    properties=[
        c2_key,
        pywbem.CIMProperty('Prop2', type='string', value='2'),
    ],
    path=c2_path,
)

a12_key1 = pywbem.CIMProperty('Ref1', type='reference', value=c1_path)
a12_key2 = pywbem.CIMProperty('Ref2', type='reference', value=c2_path)
a12_path = pywbem.CIMInstanceName(
    'TST_Association12',
    keybindings={
        a12_key1.name: a12_key1.value,
        a12_key2.name: a12_key2.value,
    },
)
a12 = pywbem.CIMInstance(
    'TST_Association12',
    properties=[
        a12_key1,
        a12_key2,
    ],
    path=a12_path,
)

conn.add_cimobjects([c1, c2, a12])